In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import pywt

# ============================
# Hybrid AOAOA Optimizer
# ============================
class HybridOptimizer:
    def __init__(self, objective_function, lower_bound, upper_bound, population_size, iterations):
        self.objective_function = objective_function
        self.lower_bound = np.array(lower_bound)
        self.upper_bound = np.array(upper_bound)
        self.population_size = population_size
        self.iterations = iterations
        self.population = np.random.uniform(low=self.lower_bound, high=self.upper_bound, size=(population_size, len(lower_bound)))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self):
        for _ in range(self.iterations):
            for i in range(self.population_size):
                # AO-based perturbation
                perturbation = np.random.uniform(-0.1, 0.1, size=self.population.shape[1])
                candidate_solution_aquila = self.population[i] + perturbation
                candidate_solution_aquila = np.clip(candidate_solution_aquila, self.lower_bound, self.upper_bound)
                fitness_aquila = self.objective_function(candidate_solution_aquila)
                if fitness_aquila < self.best_fitness:
                    self.best_fitness = fitness_aquila
                    self.best_solution = candidate_solution_aquila
            for i in range(self.population_size):
                # Arithmetic recombination
                partner_idx = np.random.randint(self.population_size)
                partner = self.population[partner_idx]
                candidate_solution_arithmetic = (self.population[i] + partner) / 2
                candidate_solution_arithmetic = np.clip(candidate_solution_arithmetic, self.lower_bound, self.upper_bound)
                fitness_arithmetic = self.objective_function(candidate_solution_arithmetic)
                if fitness_arithmetic < self.best_fitness:
                    self.best_fitness = fitness_arithmetic
                    self.best_solution = candidate_solution_arithmetic
        return self.best_solution

# ============================
# Feature Extraction
# ============================
def extract_wavelet_features(X, wavelet='db4', level=3, num_features=50):
    features = []
    for sample in X:
        coeffs = pywt.wavedec(sample, wavelet, level=level)
        flattened_coeffs = np.concatenate([c.flatten() for c in coeffs])
        features.append(flattened_coeffs[:num_features])
    return np.array(features)

def apply_pca(X, n_components=10):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=n_components)
    return pca.fit_transform(X_scaled)

def extract_combined_features(X):
    X_wavelet = extract_wavelet_features(X)
    X_pca = apply_pca(X_wavelet)
    return X_pca

# ============================
# Bi-LSTM Model Definition
# ============================
def build_lstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(50, return_sequences=True), input_shape=input_shape),
        Bidirectional(LSTM(50, return_sequences=False)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# ============================
# Training & Evaluation
# ============================
def evaluate_model(X, y):
    if X.shape[1] == 0:
        raise ValueError("No features selected! Adjust AOAOA feature selection.")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape for LSTM: (samples, timesteps, features)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    model = build_lstm_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    return mse, mae, rmse, r2, y_test, y_pred

# ============================
# Multi-Station Processing
# ============================
stations = {
    'AshokVihar': '/content/AshokVihar_Hourly.csv',
    'DCStadium': '/content/DCStadium_Hourly.csv',
    'DwarkaSec8': '/content/DwarkaSec8_Hourly.csv',
    'NehruNagar': '/content/NehruNagar_Hourly.csv',
    'Najafgarh': '/content/Najafgarh_Hourly.csv',
    'Okhla': '/content/Okhla_Hourly.csv'
}

threshold = 0.40  # Feature selection threshold
results = {}

for station, file_path in stations.items():
    print(f"\nProcessing Station: {station}")

    # Load Data
    df = pd.read_csv(file_path)

    # Preprocessing
    scaler = MinMaxScaler()
    X_full = scaler.fit_transform(df.iloc[:, :-1].values)
    y = scaler.fit_transform(df.iloc[:, -1].values.reshape(-1, 1))

    # Feature Extraction
    X_extracted = extract_combined_features(X_full)

    # Feature Selection with Hybrid AOAOA
    objective_function = lambda x: np.sum(x**2)  # Example objective, replace with your actual
    hybrid_optimizer = HybridOptimizer(
        objective_function,
        lower_bound=[-1] * X_extracted.shape[1],
        upper_bound=[1] * X_extracted.shape[1],
        population_size=50,
        iterations=100
    )
    selected_features = hybrid_optimizer.optimize()

    # Select features above threshold
    X_selected = X_extracted[:, selected_features > threshold]

    # Final Model Evaluation
    X_final = X_selected if X_selected.shape[1] > 0 else X_extracted
    mse, mae, rmse, r2, y_test, y_pred = evaluate_model(X_final, y)

    # Store results
    results[station] = {"MSE": mse, "MAE": mae, "RMSE": rmse, "R² Score": r2}

# ============================
# Print Final Results
# ============================
print("\nFinal Model Evaluation Across Stations:")
for station, metrics in results.items():
    print(f"\nStation: {station}")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")



Processing Station: AshokVihar


/usr/local/lib/python3.12/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0059 - val_loss: 0.0033
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0031 - val_loss: 0.0023
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0023 - val_loss: 0.0021
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0017 - val_loss: 0.0022
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0017 - val_loss: 0.0015
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 10/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 11/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 12/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/s

/usr/local/lib/python3.12/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Epoch 1/50


...

147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0030 - val_loss: 0.0018
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.3227e-04 - val_loss: 0.0010
Epoch 10/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.7133e-04 - val_loss: 9.2599e-04
Epoch 11/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.4287e-04 - val_loss: 8.5555e-04
Epoch 12/50
147/147 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0132 - val_loss: 0.0068
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0067 - val_loss: 0.0045
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0039 - val_loss: 0.0024
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0024 - val_loss: 0.0019
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0021 - val_loss: 0.0018
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 10/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 11/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 12/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import pywt

# ============================
# Hybrid AOAOA Optimizer
# ============================
class HybridOptimizer:
    def __init__(self, objective_function, lower_bound, upper_bound, population_size, iterations):
        self.objective_function = objective_function
        self.lower_bound = np.array(lower_bound)
        self.upper_bound = np.array(upper_bound)
        self.population_size = population_size
        self.iterations = iterations
        self.population = np.random.uniform(
            low=self.lower_bound,
            high=self.upper_bound,
            size=(population_size, len(lower_bound))
        )
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self):
        for _ in range(self.iterations):
            for i in range(self.population_size):
                perturbation = np.random.uniform(-0.1, 0.1, size=self.population.shape[1])
                candidate = np.clip(
                    self.population[i] + perturbation,
                    self.lower_bound,
                    self.upper_bound
                )
                fitness = self.objective_function(candidate)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = candidate

            for i in range(self.population_size):
                partner = self.population[np.random.randint(self.population_size)]
                candidate = np.clip(
                    (self.population[i] + partner) / 2,
                    self.lower_bound,
                    self.upper_bound
                )
                fitness = self.objective_function(candidate)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = candidate

        return self.best_solution

# ============================
# Feature Extraction
# ============================
def extract_wavelet_features(X, wavelet='db4', level=3, num_features=50):
    features = []
    for sample in X:
        coeffs = pywt.wavedec(sample, wavelet, level=level)
        flattened = np.concatenate(coeffs)
        features.append(flattened[:num_features])
    return np.array(features)

def apply_pca(X, n_components=10):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=n_components)
    return pca.fit_transform(X_scaled)

def extract_combined_features(X):
    return apply_pca(extract_wavelet_features(X))

# ============================
# Bi-LSTM Model
# ============================
def build_lstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(50, return_sequences=True), input_shape=input_shape),
        Bidirectional(LSTM(50)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# ============================
# Load & Combine Stations
# ============================
stations = {
    'AshokVihar': '/content/AshokVihar_Hourly.csv',
    'DCStadium': '/content/DCStadium_Hourly.csv',
    'DwarkaSec8': '/content/DwarkaSec8_Hourly.csv',
    'NehruNagar': '/content/NehruNagar_Hourly.csv',
    'Najafgarh': '/content/Najafgarh_Hourly.csv',
    'Okhla': '/content/Okhla_Hourly.csv'
}

dfs = []
for idx, (station, path) in enumerate(stations.items()):
    df = pd.read_csv(path)
    df['station_id'] = idx      # critical spatial feature
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
print("Combined shape:", combined_df.shape)

# ============================
# Preprocessing
# ============================
X = combined_df.iloc[:, :-1].values
y = combined_df.iloc[:, -1].values.reshape(-1, 1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# ============================
# Feature Extraction
# ============================
X_features = extract_combined_features(X_scaled)

# ============================
# AOAOA Feature Selection
# ============================
objective_function = lambda x: np.sum(x**2)
optimizer = HybridOptimizer(
    objective_function,
    lower_bound=[-1] * X_features.shape[1],
    upper_bound=[1] * X_features.shape[1],
    population_size=50,
    iterations=100
)

selected_features = optimizer.optimize()
threshold = 0.40
X_selected = X_features[:, selected_features > threshold]

X_final = X_selected if X_selected.shape[1] > 0 else X_features

# ============================
# Train / Test
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_scaled, test_size=0.2, random_state=42
)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test  = X_test.reshape(X_test.shape[0],  X_test.shape[1], 1)

model = build_lstm_model((X_train.shape[1], 1))
model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1
)

# ============================
# Evaluation
# ============================
y_pred = model.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_test, y_pred)

print("\n=== Unified Multi-Station Model Results ===")
print(f"MSE : {mse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²  : {r2:.4f}")


Combined shape: (70227, 22)


/usr/local/lib/python3.12/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


878/878 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.1262 - val_loss: 0.1066
Epoch 2/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1066 - val_loss: 0.0982
Epoch 3/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0989 - val_loss: 0.0920
Epoch 4/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0903 - val_loss: 0.0849
Epoch 5/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0848 - val_loss: 0.0790
Epoch 6/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0795 - val_loss: 0.0733
Epoch 7/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0744 - val_loss: 0.0685
Epoch 8/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0679 - val_loss: 0.0671
Epoch 9/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0643 - val_loss: 0.0613
Epoch 10/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0600 - val_loss: 0.0628
Epoch 11/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0570 - val_loss: 0.0598
Epoch 12/50
878/878 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step -